<a href="https://colab.research.google.com/github/rsaxby/NoteRNN/blob/master/Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Music Generation

In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/Colab\ Notebooks/data/music_data/music/classical_

# list files in directory
# !ls

In [0]:
# imports
from Dataset import Dataset
from generate import generate
from helper import *
from RNN import RNN
from Song import Song
from train import *
from google.colab import files

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

## Prepare the Dataset
The `Dataset` class creates a dataset from MIDI files in the directory specified by `data_dir`. Here we extract and sequentially store note, chord, and rest objects from the MIDI file stream in a list (to be fed to the network for training). Each object is also stored in a dictionary music21_objects, which we will later use to create MIDI events from using our generated notes.
Within the `create_dataset` method, we:


1.   Extract and store all note/chord/rest objects
2.   Create a dictionary to encode our unique objects
3.   Encode our list of notes to be used for training

Specify the data directory for the music files:

In [0]:
# directory where we store our music data
data_dir = '/content/drive/My Drive/Colab Notebooks/data/music_data/music/classical_/'

Create the `Dataset`, print the `.num_classes`, and retrieve the `encoded_notes` to be passed into the network.

In [4]:
# create dataset
# music21_objects_fp = data_dir+"music21_objects_quant_and_cr.pickle"
# notes_fp = data_dir+"notes_quant_and_cr.pickle"
# processed_fp = data_dir + "processed_q_and_cr.pickle"
# classical = Dataset(data_dir,notes_fp, music21_objects_fp, processed_fp)
# classical.create_dataset()
# print("Vocab size: {}".format(classical.num_classes))
# encoded_notes = classical.encoded_notes # encode notes
# classical.save_notes()

# if loading from pickle
music21_objects_fp = "music21_objects.pickle"
notes_fp = "notes.pickle"
processed_fp = "processed.pickle"
    
# create dataset from pickle
classical = Dataset(data_dir, notes_fp, music21_objects_fp, processed_fp)
classical.create_dataset_from_pickle()
print("Vocab size: {}".format(classical.num_classes))
encoded_notes = classical.encoded_notes # encode notes

Vocab size: 205


## Define Hyperparmeters and Create Dataloader

In [5]:
# Data params
# Sequence Length
sequence_length = 500  # of notes in a sequence
# Batch Size
batch_size = 128

# data loader 
train_loader = batch_data(classical.encoded_notes, sequence_length, batch_size)
print(len(train_loader))
print(len(classical.notes))
print(classical.num_classes)

86
11453
205


In [0]:
# Training parameters

# Learning Rate
learning_rate = 0.00001

# Model parameters
# Vocab size
vocab_size = classical.num_classes
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 32
# Hidden Dimension
hidden_dim = 1024
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 40

## Train the Network
We will instantiate the model and pass in our hyperparameters. Here I've already trained a model and will load it from the saved file using the `load_model` function.

In [7]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.1)
# load saved model
rnn = load_model('./save/trained_rnn')
print(rnn)
if train_on_gpu:
    rnn.cuda()

RNN(
  (embedding): Embedding(205, 32)
  (lstm): LSTM(32, 1024, num_layers=2, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.1)
  (fc1): Linear(in_features=1024, out_features=205, bias=True)
)


Choose the optimizer, criterion, number of epochs to train, and pass these along with the dataloader, model, batch_size, and number of epoch to train to the `train_rnn` function. You can also choose how verbose the output is by specifying `show_every_n_batches` and whether or not to plot loss over time with the `plot` (boolean) parameter. 

In [8]:
# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
# Number of Epochs
num_epochs = 1
# training the model
trained_rnn = train_rnn(train_loader,rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches, plot=False)

# saving the trained model
save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 1 epoch(s)...
Epoch:    1/1     Loss: 0.06623477758839727

Epoch:    1/1     Loss: 0.0649019055068493

Model Trained and Saved


## Generate Notes
Now that we have a trained model, we can generate notes from a prime. Select a note to act as the prime, then encode it and pass it to the generate function. You can specify the sequence length and the generate length.

In [0]:
prime = 'C all combinatorial (P6, I3, RI9)'
# create the dictionary to encode the notes
int2pitch = {classical.pitch2int[nt]:nt for nt in classical.pitch2int}

In [0]:
# generate notes
gen_length = 1 # modify the length to your preference
sequence_length = 50  # of notes in a sequence

generated_notes = generate(trained_rnn, classical.pitch2int[prime], int2pitch,sequence_length, gen_length, top_k=20)

## Create and Save the MIDI track
We will create an instance of the Song class, which takes in the list of generated notes, the dictionary of music21 objects, the data directory to which we should save the song, and the file name for the song. We then call the `create_song` method which will create the midi file from the notes and save the track.

In [27]:
# create song and save it
song = Song(generated_notes, classical.music21_objects, data_dir+"/generated/", "generated_classical.mid")
song.create_song()

Writing MIDI track
